# Деплоим бота на Яндекс.Облаке

*Дарья Касьяненко, НИУ ВШЭ*

Чтобы код работал автономно, его можно поместить в Cloud Functions в Yandex Cloud. Это модель serverless computing (модель предоставления серверных услуг без аренды или покупки оборудования). Подробнее про бессерверную архитектуру можно прочитать [здесь](https://habr.com/ru/articles/551298/).

## Регистрация и подготовка функции

Для начала войдем в Yandex Cloud с помощью Яндекс ID.

https://cloud.yandex.ru/

Войдите в консоль управления. Подтвердите создание своего первого облака и переходите к созданию платежного аккаунта. В консоли управления перейдите в раздел [Биллинг](https://console.cloud.yandex.ru/billing).

На странице **Аккаунты** нажмите в правом верхнем углу кнопку **Создать аккаунт**. Заполните данные (страну плательщика, имя платежного аккаунта). Предоставьте данные для создания платежного аккаунта (ФИО, номер карты, почту и телефон).

[Подробнее о платежном аккауте](https://cloud.yandex.ru/docs/getting-started/individuals/registration#qa)

Зайдите в [консоль](https://console.cloud.yandex.ru/) и нажмите Cloud Functions.

<img src="img/func.png">

Справа наверху нажмите **Создать функцию** и задайте имя для вашего будущего бота (например, telegram-bot).

<img src="img/func_name.png" width = 50% height = 50%>

Выберите язык программирования Python. Откроется редактор кода, куда мы и положим код нашего бота.

<img src="img/red.png">

## Бакет для хранения мемов

У нас будет простой бот, который будет присылать пользователю по нажатию кнопки мемы про программистов. Давайте для начала подготовим бакет с мемами (по сути облачная папка для хранения файлов – в нашем случае картинок).

Для этого в консоли выберем **Object Storage**.

<img src="img/obj_storage.png">

В правом верхнем углу нажмем **Создать бакет**. Зададим имя и выберем публичный доступ на чтение объектов. После создания зайдем внутрь бакета и загрузим наши 10 картинок-мемов.

После загрузки откройте первую картинку и скопируйте ссылку на нее.

<img src="img/bucket_link.png">

Вы получите ссылку типа `https://storage.yandexcloud.net/ИМЯБАКЕТА/ИМЯМЕМА.jpeg`

Внутри нашей программы мы будем обращаться к бакету через ссылку `https://storage.yandexcloud.net/ИМЯБАКЕТА/`, а потом добавлять к ней `рандомное число` (наши картинки названы от 1 до 10) и расширение файла `.jpeg`.

https://storage.yandexcloud.net/progmemes/

In [ ]:
# https://storage.yandexcloud.net/progmemes-3/1.jpeg

## Редактор кода

Сразу добавим файл `requirements.txt`, внутри которого запишем внешнюю библиотеку `pyTelegramBotAPI`, которая установится при сборке нашей функции.

<img src="img/req.png" height = 50% width = 50%>

Меняем код в index.py на код ниже.

In [ ]:
import telebot
from telebot import types
import json
import random

bot = telebot.TeleBot("TOKEN") # нужен токен вашего бота

def handler(event,context): # основная функция для обработки действий бота
    body = json.loads(event['body'])
    update = telebot.types.Update.de_json(body)
    bot.process_new_updates([update])

# делаем кнопки
button_start = types.KeyboardButton('Хочу смотреть мемы про программистов')
button_more = types.KeyboardButton('Неси еще')
button_stop = types.KeyboardButton('Хватит, пойду кодить')

# делаем две клавиатуры
keyboard_start = types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True)
keyboard_start.add(button_start)
keyboard_continue = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
keyboard_continue.add(button_more, button_stop)

# обрабатываем /start
@bot.message_handler(commands=['start'])
def start(message):
    start_text = "Привет! Опять прокрастинируешь?"
    bot.send_message(message.chat.id, start_text, reply_markup=keyboard_start)

# отправляем мемы
@bot.message_handler(regexp="Хочу смотреть мемы про программистов|Неси еще")
def send_meme(message):
    photo = 'https://storage.yandexcloud.net/progmemes-3/' + str(random.randint(1,10))+'.jpeg' # добавьте полученную выше ссылку бакета
    bot.send_photo(message.chat.id, photo, reply_markup=keyboard_continue)

# завершаем слать мемы
@bot.message_handler(regexp="Хватит, пойду кодить")
def bye(message):
    end_text = 'Удачи! Захочешь еще мемов, нажми /start'
    bot.send_message(message.chat.id, end_text, reply_markup=keyboard_start)

# обрабатываем другие сообщения
@bot.message_handler(content_types=["text"])
def repeat_all_messages(message):
    bot.send_message(message.chat.id, "Я тебя не понимаю" , reply_markup=keyboard_start)

Нажимаем **Создать версию**. Проверяем, наш тг-бот пока не работает :) Необходимо доделать вебхук.

## Webhook

Webhook - это URL-адрес, который передается API Telegram. Каждый раз, когда приходит новое обновление для бота, сервер Telegram отправляет это обновление на указанный URL. Аналогично происходит отправка сообщений.

Достанем URL нашей функции. Для этого внутри нашей функции telegram-bot зайдем в раздел **Обзор**.

<img src="img/func_link.png">

А теперь используем метод [`.set_webhook()`](https://core.telegram.org/bots/api#setwebhook) для нашего бота. Делаем это прямо в этой тетрадке.

In [1]:
import telebot

In [2]:
bot = telebot.TeleBot("TOKEN")

bot.remove_webhook()
bot.set_webhook("FUNCTION")

True

In [ ]:
print(bot.get_webhook_info())